# Locality Sensitive Hashing
A Python implementation for the [Introduction to Big Data Algorithms](https://sites.google.com/view/ibda-summer23/home) course at the [University of Salzburg](https://www.plus.ac.at/) in the summer semester 2023. The goal of this notebook is to compare different approaches of finding near-duplicate documents in a list of job announcements.

In [117]:
import pandas as pd

from time import perf_counter
from tqdm import tqdm

from src.shingling import get_shingles
from src.metrics import jaccard_similarity, signature_similarity
from src.min_hash import MinHash

We say that two documents are near duplicates if their Jaccard similarity is $\geq 0.8$.

In [35]:
s: float = 0.8 # similarity threshold
k: int = 10 # length of the shingles

In terms of signature length $h$ and thus also values for $r$ and $b$, we first need to observe how long the shingled documents are. Each job announcement is quite long and can easily go beyond 1000 shingles for shingle length 10. We want to to test different setups. Using the min-hash property $\mathbb P[s_1[i] = s_2[i]] = \text{sim}(D_1,D_2)$, I argue for the following values as they represent different probabilities for false positives and false negatives.

- $r=5$ and $b=20$ $\Rightarrow h=100$. 
    - Then if the similarity of two documents is $0.8$, the probability that they are NOT similar in all bands is $(1-(0.8)^5)^{20} = 0.00035$.
    - And if the similarity of two documents is $0.3$, the probability that they identical in at least one band is $1-(1-(0.3)^5)^{20} = 0.0474$.
- $r=6$ and $b=10$ $\Rightarrow h=60$. 
    - Then if the similarity of two documents is $0.8$, the probability that they are NOT similar in all bands is $(1-(0.8)^6)^{10} = 0.04783$.
    - And if the similarity of two documents is $0.3$, the probability that they identical in at least one band is $1-(1-(0.3)^6)^{10} = 0.00726$.
- $r=7$ and $b=25$ $\Rightarrow h=175$. 
    - Then if the similarity of two documents is $0.8$, the probability that they are NOT similar in all bands is $(1-(0.8)^7)^{25} = 0.00278$.
    - And if the similarity of two documents is $0.3$, the probability that they identical in at least one band is $1-(1-(0.3)^7)^{25} = 0.00545$.

In [79]:
setups: dict = {
    0: { 'r': 5, 'b': 20, 'h': 100 },
    1: { 'r': 6, 'b': 10, 'h': 60 },
    2: { 'r': 7, 'b': 25, 'h': 175 },
}

## 0. Load the Data
First, we need to load the data. To make comparisons, we use the data set `job-ads.txt` located in `data`. I also prepared a larget data set `job-ads-large.txt` where I copied each entry four times to make more meaningful performance comparisons.

In [27]:
jobs: pd.DataFrame = pd.read_csv('./data/job-ads.txt', header=None)
jobs.rename(columns = {0:'text'}, inplace = True)

# normally, makes sense to put to lowercase when processing it
jobs['text'] = jobs['text'].apply(lambda x: x.lower())

# print the head
jobs.head()

,text
0,movia spa opera da oltre un decennio nel merca...
1,cerchiamo un grafico web designer da inserire ...
2,"blog di anime, manga e videogames cerca artico..."
3,"dps soluzioni informatiche, società specializz..."
4,società operante nell'ambito della comunicazio...


We do the equivalent for the large data frame.

In [28]:
jobs_large: pd.DataFrame = pd.read_csv('./data/job-ads-large.txt', header=None)
jobs_large.rename(columns = {0:'text'}, inplace = True)
jobs_large['text'] = jobs_large['text'].apply(lambda x: x.lower())

## 1. Naive Comparison of Shingles
The first approach to search for near-duplicates will be a naive comparison of shingles. That is, we shingle each document and compare them directly using the Jaccard similarity. We find near duplicates by finding the near duplicate for each job announcement, i.e. create pairs.

In [58]:
# let's compute the shingles first
naive_shingles: list[set[str]] = jobs['text'].apply(lambda x: get_shingles(document=x, k=k, compressed=False))

Then we can go ahead and try to get pairs of (near) duplicates.

In [72]:
# start time
t1_naive: float = perf_counter()

# set of duplicates - the set makes sure that duplicates like (1,2) and (2,1) don't occur
duplicates_naive: set[frozenset[int]] = set()

# go through each row
for idx, shingle in tqdm(enumerate(naive_shingles), total=len(naive_shingles)):
    
    # compare the text with each other text
    for candidate_idx, candidate_shingle in enumerate(naive_shingles):
        sim: float = jaccard_similarity(shingle, candidate_shingle)
        if sim >= s and idx != candidate_idx:
            duplicates_naive.add(frozenset((idx, candidate_idx)))
            
# stop time
t2_naive: float = perf_counter()
elapsed_time_naive: float = t2_naive - t1_naive
print(f'Elapsed time: {elapsed_time_naive} seconds')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1530/1530 [02:13<00:00, 11.46it/s]

Elapsed time: 133.52640269599942 seconds


There can't be any false positives or false negatives since we compute the true (actual) similarity.

## 2. Minhash Comparison
We can also try to find documents by comparing their minhash signature in a pairwise manner. In other words, we only try to estimate the document's similarities.

In [95]:
compressed_shingles: list[set[int]] = jobs['text'].apply(lambda x: get_shingles(document=x, k=k))

# compute signatures for different choices of r and b
signatures: dict[list[int]] = {}

for key in setups.keys():
    print(f'Preparing minhash representation for {setups[key]}')
    
    # setup a minhash instance
    minhash: MinHash = MinHash(K=setups[key]['h'])
    
    # and compute the signature
    signatures[key] = []
    for shingle_set in compressed_shingles:
        signatures[key].append(minhash.minhash(shingle_set))

Preparing minhash representation for {'r': 5, 'b': 20, 'h': 100}
Preparing minhash representation for {'r': 6, 'b': 10, 'h': 60}
Preparing minhash representation for {'r': 7, 'b': 25, 'h': 175}


After pre-computing the min-hash signatures, we can then make pairwise comparisons of the document signatures.

In [103]:
# prepare a holding data structure for times and statistics
times: dict = {}
fp: dict = {}
fn: dict = {}

In [124]:
duplicates = {}
for key in setups.keys():
    print(f'Running minhash similarity search for {setups[key]}')
    
    # start time
    t1: float = perf_counter()

    # set of duplicates - the set makes sure that duplicates like (1,2) and (2,1) don't occur
    duplicates[key]: set[frozenset[int]] = set()

    # go through each row
    for idx, signature in tqdm(enumerate(signatures[key]), total=len(signatures[key])):

        # compare the text with each other text
        for candidate_idx, candidate_signature in enumerate(signatures[key]):
            sim: float = signature_similarity(signature, candidate_signature)
            if sim >= s and idx != candidate_idx:
                duplicates[key].add(frozenset((idx, candidate_idx)))

    # stop time
    t2: float = perf_counter()
    times[f'minhash_{key}'] = t2 - t1
    print(f'Elapsed time: {times[f"minhash_{key}"]} seconds')
    
    # compute false negatives and false positives
    fp[f'minhash_{key}'] = duplicates[key].difference(duplicates_naive)
    fn[f'minhash_{key}'] = duplicates_naive.difference(duplicates[key])
    print(f'False positives (fp): {len(fp[f"minhash_{key}"])}')
    print(f'False negatives (fn): {len(fn[f"minhash_{key}"])}')

Running minhash similarity search for {'r': 5, 'b': 20, 'h': 100}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1530/1530 [00:12<00:00, 122.99it/s]


Elapsed time: 12.443188043995178 seconds
False positives (fp): 12
False negatives (fn): 5
Running minhash similarity search for {'r': 6, 'b': 10, 'h': 60}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1530/1530 [00:06<00:00, 221.57it/s]


Elapsed time: 6.907350875997508 seconds
False positives (fp): 36
False negatives (fn): 5
Running minhash similarity search for {'r': 7, 'b': 25, 'h': 175}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1530/1530 [00:21<00:00, 72.14it/s]

Elapsed time: 21.212867485999595 seconds
False positives (fp): 7
False negatives (fn): 8
